# analysis V1 - new basis code - Dec 4, 2023
- cleaned up the data QC and organizations a bit
- having two anndata and their spin offs - both FISH and proj data 
- flip an axis (y-axis) when plotting - not in data

TODO: 
- separate and organize plotting functions 
- organize the plots and generate more insights
- focus on L2/3 cells

In [ ]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
import umap
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib

import anndata 

from scroutines import config_plots
from scroutines import basicu
from scroutines import miscu

In [ ]:
import sys
sys.setrecursionlimit(10000)
from scipy.cluster import hierarchy as sch

In [ ]:
pth_dat = '/u/home/f/f7xiesnm/project-zipursky/easifish/lt172/proc/v1/'
!ls $pth_dat

In [ ]:
# features
var_names = {
    'r1_c0': 'Sorcs3',
    'r1_c1': 'Kcnq5',
    'r1_c2': 'Chrm2',
    'r1_c4': 'Adamts2',
    
    'r2_c0': 'Kcnip3',
    'r2_c1': 'Rorb',
    'r2_c2': 'Cdh13',
    'r2_c4': 'Cntn5',
    
    'r3_c0': 'Cdh12',
    'r3_c1': 'Gria3',
    'r3_c2': 'Cntnap2',
    'r3_c4': 'Gabrg3',
    
    'r4_c0': 'Kcnh5',
    'r4_c1': 'RL Cre',
    'r4_c2': 'Slc17a7',
    'r4_c4': 'Grm8',
    
    'r5_c0': 'Ncam2',
    'r5_c1': 'Rfx3',
    'r5_c2': 'Epha10',
    'r5_c4': 'Baz1a',
}
proj_idx = np.array(['r4_c1'])

# cells
f_meta = os.path.join(pth_dat, 'roi.csv')

# spots
f_spot = os.path.join(pth_dat, 'spotcount.csv')

In [ ]:

var = pd.Series(var_names).to_frame('name')
var['proj'] = False
var.loc[proj_idx, 'proj'] = True

var_idx = var.index.values.astype(str)
var_i2n = var['name'] 
var_n2i = var.reset_index().set_index('name')['index']

gene_idx = np.array([idx for idx in var_idx if idx not in proj_idx])

# ftrs_order  = gene_names.index.values 
# genes_order = gene_names.values
var

In [ ]:
meta = pd.read_csv(f_meta, index_col=0)
spot = pd.read_csv(f_spot, index_col=0)
assert np.all(meta.index.values == spot.index.values)
print(meta.shape, spot.shape) # 

max_x, max_y, max_z = meta[['x', 'y', 'z']].describe().loc['max']
min_x, min_y, min_z = meta[['x', 'y', 'z']].describe().loc['min']

meta['to_edge'] = np.minimum(
    np.minimum(meta['x']-min_x, max_x-meta['x']), 
    np.minimum(meta['y']-min_y, max_y-meta['y']), 
    np.minimum(meta['z']-min_z, max_z-meta['z']),
)
meta['cov'] = spot.sum(axis=1)

print(f'x: {min_x:.1f}\t{max_x:.1f}')
print(f'y: {min_y:.1f}\t{max_y:.1f}')
print(f'z: {min_z:.1f}\t{max_z:.1f}')

In [ ]:
adata = anndata.AnnData(X=spot.values, obs=meta, var=var)
adata

In [ ]:
# remove outliers adata -> adata2
df = adata.obs
conds = [
    df['area'] > 500,
    df['x'] > min_x + 20,
    df['x'] < max_x - 20,
    
    df['y'] > min_y + 20,
    df['y'] < max_y - 20,
    
    df['z'] > min_z + 20,
    df['z'] < max_z - 20,
]
cond_all = np.ones(len(df)) > 0
for cond in conds:
    cond_all = np.logical_and(cond_all, cond)
    print(cond_all.sum())
print(f"Num cells before and after: {len(df)} -> {cond_all.sum()}")

# remove outliers
adata2 = adata[cond_all].copy()
adata2


In [ ]:
# separate the proj data adata2 -> adata3
adata3 = adata2[:,gene_idx].copy()
adata3.obs[proj_idx] = np.array(adata2[:,proj_idx].X)
adata3.obs['cov_gene'] = np.array(adata3.X.sum(axis=1))
adata2

In [ ]:
def norm_data(adata):
    """This procedure is independnet of each gene
    norm by size (cell volume)
    norm by log2(1+)
    norm by zscore
    """
    # size
    med_size = adata.obs['area'].median()
    max_size = adata.obs['area'].max()
    min_size = adata.obs['area'].min()

    print(f"Med cell size {med_size:.1f} um^3\t  {np.power(med_size,1/3):.1f} um")
    print(f"Max cell size {max_size:.1f} um^3\t  {np.power(max_size,1/3):.1f} um")
    print(f"Min cell size {min_size:.1f} um^3\t  {np.power(min_size,1/3):.1f} um")

    size_factor = (adata.obs['area']/med_size).values


    # norm by size; by log2+1; by zscore
    mat_raw = np.array(adata.X)
    mat_nrm = mat_raw/size_factor.reshape(-1,1) # .divide(size_factor, axis=0)
    mat_log = np.log2(1+mat_nrm)
    mat_zsc = (mat_log - np.mean(mat_log, axis=0))/np.std(mat_log, axis=0)

    adata.layers['nrm'] = mat_nrm
    adata.layers['log'] = mat_log
    adata.layers['zsc'] = mat_zsc
    
    return # mat_raw, mat_nrm, mat_log, mat_zsc

In [ ]:
norm_data(adata2)
norm_data(adata3)

In [ ]:
var_idx

In [ ]:
# cols
raw_var_idx = np.char.add(var_idx, '_raw')
nrm_var_idx = np.char.add(var_idx, '_nrm')
log_var_idx = np.char.add(var_idx, '_log')
zsc_var_idx = np.char.add(var_idx, '_zsc')

raw_gene_idx = np.char.add(gene_idx, '_raw')
nrm_gene_idx = np.char.add(gene_idx, '_nrm')
log_gene_idx = np.char.add(gene_idx, '_log')
zsc_gene_idx = np.char.add(gene_idx, '_zsc')

# binary analysis
bins_8p = np.linspace(0,400,8+1).astype(int)
bins_4p = np.linspace(0,400,4+1).astype(int)
print(bins_8p, bins_4p)

# to be used later

In [ ]:
mat_ftrs = pd.DataFrame(adata3.layers['zsc'], index=adata3.obs.index, columns=adata3.var.index)
mat_prjs = adata3.obs[proj_idx] 

In [ ]:
df_p2 = adata2.obs.copy()
df_p2[nrm_var_idx] = np.array(adata2.layers['nrm'])

df_p2['xb_8p'] = pd.cut(df_p2['x'], bins=bins_8p)
df_p2['yb_8p'] = pd.cut(df_p2['y'], bins=bins_8p)
df_p2['zb_8p'] = pd.cut(df_p2['z'], bins=bins_8p)

df_p2['xb_4p'] = pd.cut(df_p2['x'], bins=bins_4p)
df_p2['yb_4p'] = pd.cut(df_p2['y'], bins=bins_4p)
df_p2['zb_4p'] = pd.cut(df_p2['z'], bins=bins_4p)

In [ ]:
df_p3 = adata3.obs.copy()
df_p3[nrm_gene_idx] = np.array(adata3.layers['nrm'])

df_p3['xb_8p'] = pd.cut(df_p3['x'], bins=bins_8p)
df_p3['yb_8p'] = pd.cut(df_p3['y'], bins=bins_8p)
df_p3['zb_8p'] = pd.cut(df_p3['z'], bins=bins_8p)

df_p3['xb_4p'] = pd.cut(df_p3['x'], bins=bins_4p)
df_p3['yb_4p'] = pd.cut(df_p3['y'], bins=bins_4p)
df_p3['zb_4p'] = pd.cut(df_p3['z'], bins=bins_4p)

# report 

In [ ]:
for idx in var_idx:
    val = adata[:,idx].X[:,0]
    print(f'{idx}\t{var_i2n.loc[idx]}\t{np.min(val):.1f}\t{np.max(val):.1f}\t{100*np.sum(val>0)/len(val):.2f}%')

In [ ]:
cols = ['x', 'y', 'z', 'area']
with sns.plotting_context('paper'):
    fig, axs = plt.subplots(4, 1, figsize=(1*6, 2*4))
    for ax, col in zip(axs, cols):
        sns.histplot(adata.obs[col], ax=ax)
        ax.set_xlabel(col)
    fig.subplots_adjust(hspace=0.5)
    plt.show()


In [ ]:
sns.scatterplot(data=adata.obs, x='area', y='cov', s=2, edgecolor='none')

In [ ]:
with sns.plotting_context('paper'):
    fig, axs = plt.subplots(2, 1, figsize=(10*1,4*2))
    ax = axs[0]
    sns.boxplot(data=adata.X, ax=ax)
    ax.set_xticklabels(adata.var.index.values, rotation=90) 
    ax.set_ylabel('counts')
    ax.set_xlabel('Genes')
    sns.despine(ax=ax)

    ax = axs[1]
    sns.boxplot(data=adata.X, ax=ax)
    ax.set_xticklabels(adata.var['name'].values, rotation=90)
    sns.despine(ax=ax)
    ax.set_ylim([0,50])
    ax.set_ylabel('counts')
    ax.set_xlabel('Genes')

In [ ]:
sns.scatterplot(data=adata3.obs, x='area', y='cov', s=2, edgecolor='none')
sns.scatterplot(data=adata3.obs, x='area', y='cov_gene', s=2, edgecolor='none')

In [ ]:
sns.scatterplot(data=adata3.obs, x='area', y=proj_idx[0], s=2, edgecolor='none')

In [ ]:
# # PCAs
# n_pc = len(genes_order)
# idx_pcs = np.arange(1, n_pc+1)

# pca = PCA(n_components=n_pc)
# pcs = pca.fit_transform(mat_zscore) 
# plt.plot(idx_pcs, pca.explained_variance_ratio_, 'o')
# plt.xticks(idx_pcs)
# plt.ylim(ymin=0)
# plt.axhline(1/n_pc, color='k')
# plt.xlabel('PC')
# plt.ylabel('Explained variance')

# for i in idx_pcs:
#     dfnew[f'PC{i}'] = pcs[:,i-1]

In [ ]:
# # UMAPs
# Zspace = umap.UMAP(n_components=2).fit_transform(pcs)
# dfnew['umap_x'] = Zspace[:,0]
# dfnew['umap_y'] = Zspace[:,1]

# # UMAPs more
# nns = [30, 60, 120, 240, 480]
# for nn in nns:
#     Zspace = umap.UMAP(n_components=2, n_neighbors=nn).fit_transform(pcs)
#     dfnew[f'umap_x_nn{nn}'] = Zspace[:,0]
#     dfnew[f'umap_y_nn{nn}'] = Zspace[:,1]

# # clsts
# numclsts = [2, 5, 10]
# for k in numclsts: 
#     model = KMeans(n_clusters=k)
#     clsts = model.fit_predict(pcs)
#     dfnew[f'clsts_k{k}'] = np.char.add("c", clsts.astype(str))

# # clsts
# G = miscu.build_feature_graph_knnlite(pcs, k=15, metric='euclidean')
# rs = [0.01, 0.02, 0.05, 0.1, 1]
# for r in rs:
#     clsts = miscu.leiden(G, dfnew.index.values, resolution=r) 
#     dfnew[f'clsts_r{r}'] = np.char.add("c", np.array(clsts).astype(str))

# check basics
- y: pia-ventricular axis

# z-sectioning visuals
- bin into zbin
- plot for each zbin

In [ ]:
def gen_palette(dfnew, col):
    # set global colorbar
    hue_order = np.unique(dfnew[col]) 
    ncat = len(hue_order)
    if ncat > 5:
        cmap = 'Paired'
    else:
        cmap = 'tab10'
    palette = {clst: clr for clst, clr in zip(hue_order, sns.color_palette(cmap, ncat))}
    return palette

In [ ]:
def view_z_sections(dfnew, sp_x, sp_y, col=None, cmap='coolwarm', title=None, vmaxp=95, vminp=0, flip_y=False):
    """
    """
    # assert len(dfnew['bins_zb'].unique())-1 == 8 # 8 intervals
    
    # set global colorbar
    if col is not None:
        vmax = np.percentile(dfnew[col].values, vmaxp)
        vmin = -vmax*0.1
        # vmin = np.percentile(dfnew[col].values, vminp)
        
    fig, axs = plt.subplots(2, 4, figsize=(4*4,2*4), sharex=True, sharey=True)
    if flip_y: 
        axs.flat[0].invert_yaxis()
    for i, (thisbin, dfsub) in enumerate(dfnew.groupby('zb_8p')):
        ax = axs.flat[i]
        ax.grid(False)

        x = dfsub[sp_x].values
        y = dfsub[sp_y].values
        if col is not None:
            c = dfsub[col].values
            g = ax.scatter(x, y, c=c, s=10, cmap=cmap, vmax=vmax, vmin=vmin,
                           edgecolor='none', 
                          )
        else:
            g = ax.scatter(x, y, s=10, c='gray', 
                           edgecolor='none', 
                          )
            
            
        sns.despine(ax=ax)
        ax.set_aspect('equal')
        ax.set_title(f" z: {thisbin}", loc='left', y=0.95)#va='top')#va='top', y=0.96)

        if i%4 == 0:
            ax.set_ylabel(sp_y)
        if i >= 4:
            ax.set_xlabel(sp_x)

    if col is not None:
        cax = fig.add_axes([0.93, 0.3, 0.01, 0.3])
        fig.colorbar(g, cax=cax, label='', shrink=0.3, aspect=10, ticks=[0,vmax], orientation='vertical')
    fig.subplots_adjust(hspace=0.1, wspace=0.05)
    if title is not None:
        fig.suptitle(title, fontsize=20, y=0.92)
    plt.show()

In [ ]:
def view_z_sections_4panels(dfnew, sp_x, sp_y, col=None, cmap='coolwarm', title=None, vmaxp=95, vminp=0, flip_y=False):
    """
    """
    # assert len(dfnew['bins_zb_4p'].unique())-1 == 4 # 4 intervals
    
    # set global colorbar
    if col is not None:
        vmax = np.percentile(dfnew[col].values, vmaxp)
        vmin = -vmax*0.1
        # vmin = np.percentile(dfnew[col].values, vminp)
        
    fig, axs = plt.subplots(1, 4, figsize=(4*4,1*4), sharex=True, sharey=True)
    if flip_y: 
        axs.flat[0].invert_yaxis()
    for i, (thisbin, dfsub) in enumerate(dfnew.groupby('zb_4p')):
        ax = axs.flat[i]
        ax.grid(False)

        x = dfsub[sp_x].values
        y = dfsub[sp_y].values
        if col is not None:
            c = dfsub[col].values
            g = ax.scatter(x, y, c=c, s=8, cmap=cmap, vmax=vmax, vmin=vmin,
                           edgecolor='none', 
                          )
        else:
            g = ax.scatter(x, y, s=8, c='gray', 
                           edgecolor='none', 
                          )
            
        sns.despine(ax=ax)
        ax.set_aspect('equal')
        ax.set_title(f" z: {thisbin}", loc='left', y=0.95)#va='top')#va='top', y=0.96)

        if i == 0:
            ax.set_ylabel(sp_y)
        ax.set_xlabel(sp_x)

    if col is not None:
        cax = fig.add_axes([0.93, 0.3, 0.01, 0.3])
        fig.colorbar(g, cax=cax, label='', shrink=0.3, aspect=10, ticks=[0,vmax], orientation='vertical')
    fig.subplots_adjust(hspace=0.1, wspace=0.05)
    if title is not None:
        fig.suptitle(title, fontsize=20, y=0.92)
    plt.show()

In [ ]:
def view_z_sections_labels(dfnew, sp_x, sp_y, col, palette=None, title=None, flip_y=False):
    """
    """
    # assert len(dfnew['bins_zb'].unique())-1 == 8 # 8 intervals
    
    # set global colorbar
    hue_order = np.unique(dfnew[col]) 
    ncat = len(hue_order)
    if ncat > 5:
        cmap = 'Paired'
    else:
        cmap = 'tab10'
    if palette is None:
        palette = {clst: clr for clst, clr in zip(hue_order, sns.color_palette(cmap, ncat))}
        
    fig, axs = plt.subplots(2, 4, figsize=(4*4,2*4), sharex=True, sharey=True)
    if flip_y: 
        axs.flat[0].invert_yaxis()
    for i, (thisbin, dfsub) in enumerate(dfnew.groupby('zb_8p')):
        ax = axs.flat[i]
        ax.grid(False)
        if i == 4-1:
            legend = True
        else:
            legend = False
        g = sns.scatterplot(data=dfsub, x=sp_x, y=sp_y, hue=col, 
                            hue_order=hue_order, palette=palette,
                            s=10, 
                            edgecolor='none', 
                            ax=ax,
                            legend=legend,
                          )
        sns.despine(ax=ax)
        ax.set_aspect('equal')
        ax.set_title(f" z: {thisbin}", loc='left', y=0.95)#va='top')#va='top', y=0.96)

        if i%4 == 0:
            ax.set_ylabel(sp_y)
        if i >= 4:
            ax.set_xlabel(sp_x)
        if i == 4-1:
            ax.legend(bbox_to_anchor=(1,1))

    fig.subplots_adjust(hspace=0.1, wspace=0.05)
    if title is not None:
        fig.suptitle(title, fontsize=20, y=0.92)
    plt.show()

In [ ]:
sp_x, sp_y = 'x', 'y'
col = gene_idx[0] # ftrs_order[0]
title = var_i2n[col]

view_z_sections(df_p2, sp_x, sp_y, col=col+'_nrm', cmap='Greys', title=title, flip_y=True)
view_z_sections_4panels(df_p2, sp_x, sp_y, col=col+'_nrm', cmap='Greys', title=title, flip_y=True)

# hierarchical clustering

In [ ]:
def gen_discrete_colors(N):
    """
    """
    fig, ax = plt.subplots(figsize=(6, 1))
    fig.subplots_adjust(bottom=0.5)
    cmap = matplotlib.cm.jet
    norm = matplotlib.colors.BoundaryNorm(1+np.arange(N), cmap.N, extend='max')
    dismap = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)
    fig.colorbar(dismap,
                 cax=ax, orientation='horizontal')
    plt.show()
    
    palette = {i: dismap.to_rgba(i)[:3] for i in 1+np.arange(N)}
    plt.figure(figsize=(5,1))
    plt.scatter(1+np.arange(N), [0]*N, c=[palette[p] for p in palette.keys()])
    plt.show()
    return palette, dismap

In [ ]:
# sch
col_link = sch.linkage(mat_ftrs.T, method='ward', metric='euclidean')
col_order = sch.dendrogram(col_link, no_plot=True)['leaves']

row_link = sch.linkage(mat_ftrs, method='ward', metric='euclidean')
row_order = sch.dendrogram(row_link, p=30, truncate_mode='none', no_plot=True)['leaves']

# get clusters
clsts1 = sch.fcluster(row_link, 100, criterion='distance')
clsts2 = sch.fcluster(row_link,  50, criterion='distance')
clsts3 = sch.fcluster(row_link,  30, criterion='distance')

print(clsts1.shape, np.unique(clsts1).shape)
print(clsts2.shape, np.unique(clsts2).shape)
print(clsts3.shape, np.unique(clsts3).shape)


# register
df_p3['clst_l1'] = clsts1
df_p3['clst_l2'] = clsts2
df_p3['clst_l3'] = clsts3

df_p2['clst_l1'] = clsts1
df_p2['clst_l2'] = clsts2
df_p2['clst_l3'] = clsts3

# colors
palette1, dismap1 = gen_discrete_colors(len(np.unique(clsts1)))
palette2, dismap2 = gen_discrete_colors(len(np.unique(clsts2)))
palette3, dismap3 = gen_discrete_colors(len(np.unique(clsts3)))

In [ ]:
fig = plt.figure(figsize=(10,15))
axd = fig.subplot_mosaic("A"*8+"BCDEF")
ax = axd["A"]
sns.heatmap(
    mat_ftrs.iloc[row_order,col_order],
    yticklabels=False,
    cmap='coolwarm',
    cbar_kws=dict(shrink=0.5, location='bottom', label='zscore log2(1+normed counts)'),
    vmax=3, vmin=-3,
    ax=ax,
)
ax.set_ylabel('Cells')

for i, (ax, col) in enumerate(zip(
    (axd["B"], axd["C"], axd["D"]),
    ('clst_l1', 'clst_l2', 'clst_l3'),
    )):
    vec = df_p3[col].values
    sns.heatmap(
        df_p3[[col]].rename(columns={col: col.replace('clst_l', 'L')}).iloc[row_order],
        yticklabels=False,
        cmap='jet',
        cbar_kws=dict(location='bottom', aspect=5, label=''),
        ax=ax,
    )
    ax.set_ylabel('')
    positions = np.cumsum(np.unique(vec, return_counts=True)[1])
    ax.hlines(positions, xmin=-0.2, xmax=1, linewidth=1, linestyle='-', color='k', clip_on=False)
    for i, pos in enumerate(positions):
        ax.text(0, pos, i+1, color='white')

ax = axd["E"]
sns.heatmap(
    mat_prjs.iloc[row_order],
    yticklabels=False,
    cmap='Greys',
    cbar_kws=dict(location='bottom', aspect=5, label='log2(1+normed counts)'),
    ax=ax,
)
ax.set_ylabel('')

ax = axd["F"]
sns.heatmap(
    df_p3[['y']].iloc[row_order], 
    yticklabels=False,
    cmap='Greys',
    cbar_kws=dict(location='bottom', aspect=5, label='um'),
    # vmin=0,
    ax=ax,
)
ax.set_ylabel('')
fig.tight_layout()

In [ ]:
sp_x, sp_y = 'x', 'y'
fig, axs = plt.subplots(1,3,figsize=(4*3,5*1), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
# cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
for i, (col, palette, dismap, ax) in enumerate(zip(['clst_l1', 'clst_l2', 'clst_l3'], 
                                                   [ palette1,  palette2,  palette3],
                                                   [ dismap1,   dismap2,   dismap3],
                                                   axs.flat)):
    x = df_p3[sp_x].values
    y = df_p3[sp_y].values
    c = df_p3[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=pd.Series(palette)[c], s=5, edgecolor='none',)
    sns.despine(ax=ax)
    ax.set_title(col)
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    fig.colorbar(dismap, ax=ax, orientation='horizontal', shrink=0.5, aspect=10)
    
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'x', 'y'
col = 'clst_l1'
palette = palette1
dismap = dismap1
n = len(df_p3[col].unique())
nx = min(6,n)
ny = int((n+nx-1)/nx)
suptitle = 'L1'

fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
for i, ax in enumerate(axs.flat):
    x = df_p3[sp_x].values
    y = df_p3[sp_y].values
    c = df_p3[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=[palette[_c] if i+1==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)
    
    sns.despine(ax=ax)
    ax.set_title(f"C{i+1}")
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    
fig.suptitle(suptitle)
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'x', 'y'
col = 'clst_l2'
palette = palette2
dismap = dismap2
n = len(df_p3[col].unique())
nx = min(4,n)
ny = int((n+nx-1)/nx)
suptitle = 'L2'

fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
for i, ax in enumerate(axs.flat):
    x = df_p3[sp_x].values
    y = df_p3[sp_y].values
    c = df_p3[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=[palette[_c] if i+1==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)
    
    sns.despine(ax=ax)
    ax.set_title(f"C{i+1}")
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    
fig.suptitle(suptitle)
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'x', 'y'
fig, axs = plt.subplots(5,4,figsize=(4*4,4*5), sharey=True, sharex=True)
cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
axs.flat[0].invert_yaxis()
for i, (col, ax) in enumerate(zip(var_idx, axs.flat)):
    x = df_p2[sp_x].values
    y = df_p2[sp_y].values
    c = df_p2[col+'_nrm'].values
    vmax=np.percentile(c, 95)
    
    ax.axis('off')
    ax.grid(False)
    g = ax.scatter(x, y, c=c/vmax, s=5, edgecolor='none', cmap='Greys', vmax=1, vmin=-0.1) #vmax, vmin=-0.1*vmax)
    sns.despine(ax=ax)
    ax.set_title(var_i2n[col])
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    
fig.colorbar(g, cax=cbar_ax, label='Normed counts\n(0-95 perctl.)', aspect=5, shrink=0.3, ticks=[0, 1])
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
group = 'clst_l2'
dfmean = df_p2.groupby(group)[np.char.add(var_idx, '_nrm')].mean()
dfmean = basicu.zscore(dfmean, axis=0)

fig, ax = plt.subplots(figsize=(6,4))
sns.heatmap(dfmean.T, ax=ax, 
            yticklabels=var_i2n[var_idx], 
            cmap='coolwarm', cbar_kws=dict(label='zscore mean exp.', shrink=0.5, aspect=10))

# co-expression, clustering, cluster enrichment of expression

In [ ]:
sp_x, sp_y = 'x', 'y'
col = 'clst_l1'
view_z_sections_labels(df_p3, sp_x, sp_y, col, palette=palette1, title=None, flip_y=True)